# Algorithm Ideas

# **ROUGH DRAFT ALL CODE IS IN PROGRESS MY WORKSTYLE IS SLOPPY I KNOW**


In [5]:
#includes
import math
import numpy as np
from matplotlib import pyplot as plt
import librosa
import sys
from scipy import signal
import scipy.io.wavfile as wav
from scipy.fft import fft, ifft
import contextlib
import os
import soundfile as sf


## Audio
### Voice recording phase: the system records voice at 5-s intervals.

### Recorded data are bandpass filtered (50 Hz~3000 Hz)

### Data are filtered with a Wiener filter. 
The Wiener filter minimizes the Mean Square Error (MSE) between the estimated random process and the desired operation. This filter is generally used to remove noise from a recorded voice.
### Short sounds and background noise are removed.
 First, an adaptive threshold to remove background noise has been used. The reference level of environmental noise must be calculated. As the noise in the disaster area is high and highly variable, an adaptive background noise reference has been defined according to the equation:
$$Ref_{noise}  = αVol_t + (1 − α)Vol_{t-1}$$
where α is the smoothing factor of Refnoise change, Volt is the average volume [dB] of current 5 s voice data, Volt−1 is the volume of previous 5 s voice data. It has been empirically found that a = 30% yields the best performance. Then, if the volume of the sound sample is lower than 1.3 times Refnoise, the algorithm identifies the sound sample as environmental noise and discards it. Sound signals that are 1.3 times higher than Refnoise are suspect sounds. Then, the algorithm checks the length of this suspect sound. As human voice sound is assumed to last more than 300 ms, sounds shorter than 300 ms are removed. After removing short sounds, this suspect sound is processed with SVM to identify possible human noise.

In [6]:
# Preprocessing

#Band-Pass

def bandpass(wavname):
        sample_rate, wav_file = wav.read(wavname)

        low = 50
        high = 3000

        nyq = 0.5*sample_rate
        lowpass = low/nyq
        highpass = high/nyq
        b, a = signal.butter(3, [lowpass, highpass], btype="band")
        print(b)
        print(wav_file)
        filtered_wav = signal.filtfilt(b, a, wav_file, padtype=None)
        print(filtered_wav)

        wav.write(str(wavname+"_filtered.wav"), sample_rate, filtered_wav.astype("float32"))
        return str(wavname+"_filtered.wav")




In [ ]:
def wiener_filter(wavdata):
    return signal.wiener(wavdata)

In [ ]:
def short_sound_removal(wavdata, pwa_volume, smoothing_factor):
    # pwa_volume -- average volume of the previous 5sec clip.
    # smoothing factor -- float between 0 and 1, 0.3 is empirically tested to be best
    volume = sum(wavdata)/len(wavdata)
    refnoise = smoothing_factor*volume - (1-smoothing_factor)
    
    removed_ambient = [0 if x < refnoise else x for x in wavdata]

    # Then scan for segments greater than 300ms of nonzeros
    # PENDING, DIVIDING CLIPS BY LENGTH IS GONNA BE A LITTLE WEIRD I THINK
    final_wav = 1 # replace once figured out
    return final_wav




### Segmentation.
 The 5-s audio signal, after removing short sounds and background noise, is broken into shorter audio samples of 10 ms.
### Audio statistical features are computed for these shorter 10-ms audio samples.
### SVM Classification.
 Sounds are differentiated in human voice or noise.

In [ ]:
def splitter(wavname):
    # read
    sample_rate, wavfile = wav.read(wavname)
    # if length is not exactly 5 seconds, crop from both ends until so, or pad.
    
    # Set the length of each clip in milliseconds
    clip_length = 10
    clip_frames = int(clip_length * sample_rate / 500)

    
    # Create a directory to store the output clips
    # Not sure if we will want this, but depending on whether we use an SVM or something else may want this temp stored somewhere for
    # other applications.
    if not os.path.exists('output'):
        os.makedirs('output')

    # Read and process each clip from the WAV file
    clip_num = 0
    while clip_num:
        # Read the next clip from the WAV file
        clip_frames_raw = wavfile.readframes(clip_frames * num_channels)
        if not clip_frames_raw:
            break
        clipped_frames.append(clip_frames_raw)

        # Split the frames into channels and write the clip to a new WAV file
        # clip_frames_array = wavfile.struct.unpack("%dh" % (clip_frames * num_channels), clip_frames_raw)
        # clip_frames_channels = [clip_frames_array[ch::num_channels] for ch in range(num_channels)]
        wav.write('output/clip_{}.wav'.format(clip_num), sample_rate)
        # Increment the clip counter
        clip_num += 1

    return clipped_frames
    ## Currently writes all the clips to a separate directory, can change if we want to handle everything in one program


error: unpack requires a buffer of 3840 bytes

### Energy Entropy

#### $$H(X) = −\Sigma^{n}_{i=1}p(x_i)\log{p(x_i)}$$

where X is a discrete RV with pdf $p(x)$ and readings $\{x_i\}_{i=1}^n$

In [ ]:
def compute_entropy(hist): 
    # Calculate the entropy of the histogram
    return -np.sum(hist * np.log2(hist + 1e-6)) #Safeguard value to prevent log of 0

### Signal Energy

#### $$E_s = \int^{\infty}_{-\infty}|x(t)|^2dt$$

Where $x(t)$ is a continuous-time signal



In [ ]:
## May need to look into this one more -- Not sure if applying correctly
## Basically square all values, sum them up, divide by amount of values for a discrete integral instead of continuous
def compute_signal_energy(histogram):
    signal_energy = np.sum(np.square(histogram))/len(histogram)
    return signal_energy

### Zero Crossing Rate

#### $$ZCR = \frac{1}{T-1}\Sigma^{T-1}_{t=1}1_{R>0}(s_ts_{t-1})$$

where $s$ is a voice signal of length $T$ and $1_{R>0}$ is an indicator function


In [ ]:
def compute_zero_cross(clip):
    # Set indicator function values of main hist
    signage = np.sign(clip)
    #Find sum
    total = np.sum(np.abs(np.diff(signage)))
    return total/((len(clip)-1))


### Spectral Roll-Off

#### $$\Sigma^{R_t}_{n=1}M_t[n] = 0.85 \times \Sigma^{N}_{n=1}M_t[n]$$

Where $M_t[n]$ is the magnitude of the Fourier Transform at frame $t$ and frequnecy bin $n$ and $R_t$ is the frequency

In [ ]:
def compute_spectral_rolloff(fourier, sample_rate):
    return librosa.feature.spectral_rolloff(y=fourier, sr=sample_rate, n_fft=64)

### Spectral Centroid

#### $$C = \frac{\Sigma^{N-1}_{n=0}f(n)x(n)}{\Sigma^{N-1}_{n=0}x(n)}$$

where $x(n)$ represents the weighted frequency value, or magnitude, of bin $n$, and $f(n)$ represents the center frequency of that bin

In [ ]:
def compute_spectral_centroid(fourier, sample_rate):
    return librosa.feature.spectral_rolloff(y=fourier, sr=sample_rate, n_fft=64)

### Spectral Flux

#### $$F_t = \Sigma^{N}_{n=1}(N_t[n]-N_{t-1}[n])^2$$

where $N_t[n]$ and $N_{t-1}[n]$ are the normalized Fourier transform at frames t and t-1

In [ ]:
def compute_spectral_flux(n_fourier):
    #TODO

### Then, a driver that gathers all the information for the clips

For each 5s clip, divide into 1000 5ms clips.
Need to discuss whether we are evaluating 1 5s clip or 1000 5ms clips for analysis.  I vote 1 5s clip with 6 attributes

In [7]:
def collect_data(wavname):
    #filter the audio bandpass
    bandpass_wav_file = bandpass(wavname)
    #Wiener filter
    wiener = wiener_filter(bandpass_wav_file)
    #Remove short sounds
    
    #Preprocessing of original audio complete, begin splitting and feature definition.
    splits = splitter(filtered_wav_file)
    rate, data = wav.read(filtered_wav_file)
    fourier = fft(data)
    min_fourier = min(fourier)
    max_fourier = max(fourier)
    normalized_fourier = [(x-min_fourier/(max_fourier-min_fourier)) for x in fourier]

    features = np.array() #stores for each clip
    # Call codee above for splitting wav data into clips
    i=0
    for clip in clips:
        # Calculate the decibel levels for the clip
        db = librosa.amplitude_to_db(np.abs(clip), ref=np.max)
        # Calculate the histogram of the decibel levels
        hist, bins = np.histogram(db, bins='auto', density=True)
        #Entropy - can be performed on histogram
        features[i][0] = compute_entropy(hist)
        #Signal Energy - can be performed on histogram
        features[i][1] = compute_signal_energy(hist)
        #Zero Crossing Rate - performed on audio
        features[i][2] = compute_zero_cross(hist)
        #Spectral Roll Off
        features[i][3] = compute_spectral_rolloff(fourier)
        #Spectral Centroid
        features[i][4] = compute_spectral_centroid(fourier)
        #Spectral Flux
        features[i][5] = compute_spectral_flux(normalized_fourier)
    
    # For entropy, ZCT, Roll Off, Centroid, compute SD
    
    # For Signal Energy, Flux, compute SD by Squared Mean
    

    #Features prepped for SVM
    return collected_data
        


# SVM Classification
Going to have to train an SVM somehow.  I think the best way to do it would be to record a long wav on our current mic setup, split it, mark which sections had a voice and which ones did not (i.e. set it up in the living room for a bit).  Then listen and manually track voices.  After that, use as training data.  If we are measuring 5 second clips, we are going to want perhaps 30 minutes of volume input.  Can also look for an SVM model online that is built or some voice data, I'm sure it exists out there.  But the model might work better if it was built on the audio from our mic.


In [ ]:
# Build SVM Model

# For REFNOISE -- Take ambient, compute average, use that for threshold of previous 5 second clip.

In [ ]:
# Run snippet through SVM Model

## $CO_2$

## LIDAR

## Thermal Camera (?)

### Helpful Links
